In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from difflib import get_close_matches, SequenceMatcher
from sklearn.metrics import mutual_info_score
from sklearn.metrics import precision_score, recall_score, f1_score
from random import choice, randrange
import requests
import time
import string
import random
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import json
import nltk
nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import words
from nltk.corpus import stopwords
!pip install wikipedia
import wikipedia
!pip install pyinterval
import os
import tarfile
import time
import pickle

[nltk_data] Downloading package words to
[nltk_data]     /Users/enzoveltri/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/enzoveltri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/enzoveltri/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
## data prep
ps = PorterStemmer()
stemmed_words_list = [ps.stem(w) for w in words.words()]

In [3]:
## utils funct
def saveCache(fileName, cache):
    tempDict = {}
    for key, value in cache.items():
        tempDict[key] = list(value)
    with open(fileName, 'w') as fp:
        json.dump(tempDict, fp)
        
def loadCache(fileName):
    data = None
    with open(fileName, 'r') as fp:
        data = json.load(fp)
    cache = {}
    for key, value in data.items():
        cache[key] = set(value)
    return cache

def findFromCached(cachedAlias, cachedProvenance, columns):
    aliasReturn = {}
    provenanceReturn = {}
    columnsToSearch = set()
    for column in columns:
        if column in cachedAlias:
            aliasReturn[column] = cachedAlias[column]
        else:
            columnsToSearch.add(column)
        if column in cachedProvenance:
            provenanceReturn[column] = cachedProvenance[column]
        else:
            columnsToSearch.add(column)
     
    return aliasReturn, provenanceReturn, columnsToSearch

In [4]:
## caches
cacheSynonym = {}
cacheRelatedTo = {}
cacheIsA = {}
cacheDerivedFrom = {}
cacheWikipedia = {}
cachedAlias = {}
cachedProvenance = {}

In [6]:
## load caches
cacheSynonym = loadCache('./cacheSynonym-small.json')
cacheRelatedTo = loadCache('./cacheRelatedTo-small.json')
cacheIsA = loadCache('./cacheIsA-small.json')
cacheDerivedFrom = loadCache('./cacheDerivedFrom-small.json')
cacheWikipedia = loadCache('./cacheWikipedia-small.json')
cachedAlias = loadCache('./cacheAlias.json')
#cachedProvenance = loadCache('./cacheProvenance.json')

In [7]:
a_file = open("cacheProvenance.pkl", "rb")
cachedProvenance = pickle.load(a_file)

In [8]:
def getLCS(string1, string2):
    match = SequenceMatcher(None, string1, string2).find_longest_match(0, len(string1), 0, len(string2))
    return string1[match.a: match.a + match.size]
    #return string2[match.b: match.b + match.size]

## CONCEPTNET.IO ##
def getSynonym(word, limit=10, dropWord=True, useCache=True):
    processedText = word.lower().replace(' ', '_')
    synonyms = set()
    if (len(processedText) <= 1):
        return synonyms, True
    if useCache and word in cacheSynonym:
        return cacheSynonym[word], True
    baseUrl = "https://api.conceptnet.io/query?node=/c/en/$WORD$&other=/c/en&limit=$LIMIT$&rel=/r/Synonym"
    processedURL = baseUrl.replace("$WORD$", processedText)
    processedURL = processedURL.replace("$LIMIT$", str(limit))
    #print("Processed URL: ", processedURL)
    obj = None
    try:
        obj = requests.get(processedURL).json()
    except Exception:
        return synonyms, False
    #obj = requests.get(processedURL).json()
    if 'edges' not in obj:
        return synonyms, False
    resultsLen = len(obj['edges'])
    edges = obj['edges']
    #print("Results:", resultsLen)
    
    for i in range(0, resultsLen):
        edge = edges[i]
        label = edge['end']['label']
        synonyms.add(label.lower())
        label = edge['start']['label']
        synonyms.add(label.lower())
    if dropWord:
        synonyms.discard(word.lower())
    if useCache:
        cacheSynonym[word] = synonyms
    return synonyms, False

def getRelatedTo(word, limit=10, dropWord=True, useCache=True):
    processedText = word.lower().replace(' ', '_')
    relatedTo = set()
    if (len(processedText) <= 1):
        return relatedTo, True
    if useCache and word in cacheRelatedTo:
        return cacheRelatedTo[word], True
    baseUrl = "https://api.conceptnet.io/query?node=/c/en/$WORD$&other=/c/en&limit=$LIMIT$&rel=/r/RelatedTo"
    processedURL = baseUrl.replace("$WORD$", processedText)
    processedURL = processedURL.replace("$LIMIT$", str(limit))
    #print("Processed URL: ", processedURL)
    obj = None
    try:
        obj = requests.get(processedURL).json()
    except Exception:
        return relatedTo, False
    #obj = requests.get(processedURL).json()
    if 'edges' not in obj:
        return relatedTo, False
    resultsLen = len(obj['edges'])
    edges = obj['edges']
    #print("Results:", resultsLen)
    for i in range(0, resultsLen):
        edge = edges[i]
        label = edge['end']['label']
        relatedTo.add(label.lower())
        label = edge['start']['label']
        relatedTo.add(label.lower())
    if dropWord:
        relatedTo.discard(word.lower())
    if useCache:
        cacheRelatedTo[word] = relatedTo
    return relatedTo, False

def getIsA(word, limit=10, dropWord=True, useCache=True):
    processedText = word.lower().replace(' ', '_')
    isA = set()
    if (len(processedText) <= 1):
        return isA, True
    if useCache and word in cacheIsA:
        return cacheIsA[word], True
    baseUrl = "https://api.conceptnet.io/query?node=/c/en/$WORD$&other=/c/en&limit=$LIMIT$&rel=/r/IsA"
    processedURL = baseUrl.replace("$WORD$", processedText)
    processedURL = processedURL.replace("$LIMIT$", str(limit))
    #print("Processed URL: ", processedURL)
    obj = None
    try:
        obj = requests.get(processedURL).json()
    except Exception:
        return isA, False
    #obj = requests.get(processedURL).json()
    if 'edges' not in obj:
        return isA, False
    resultsLen = len(obj['edges'])
    edges = obj['edges']
    #print("Results:", resultsLen)
    for i in range(0, resultsLen):
        edge = edges[i]
        label = edge['end']['label']
        isA.add(label.lower())
    if dropWord:
        isA.discard(word.lower())
    if useCache:
        cacheIsA[word] = isA
    return isA, False

def getDerivedFrom(word, limit=10, dropWord=True, useCache=True):
    processedText = word.lower().replace(' ', '_')
    derivedFrom = set()
    if (len(processedText) <= 1):
        return derivedFrom, True
    if useCache and word in cacheDerivedFrom:
        return cacheDerivedFrom[word], True
    baseUrl = "https://api.conceptnet.io/query?node=/c/en/$WORD$&other=/c/en&limit=$LIMIT$&rel=/r/DerivedFrom"
    processedURL = baseUrl.replace("$WORD$", processedText)
    processedURL = processedURL.replace("$LIMIT$", str(limit))
    #print("Processed URL: ", processedURL)
    obj = None
    try:
        obj = requests.get(processedURL).json()
    except Exception:
        return derivedFrom, False
    #obj = requests.get(processedURL).json()
    if 'edges' not in obj:
        return derivedFrom, False
    resultsLen = len(obj['edges'])
    edges = obj['edges']
    #print("Results:", resultsLen)
    for i in range(0, resultsLen):
        edge = edges[i]
        label = edge['end']['label']
        if (label != word):
            derivedFrom.add(label.lower())
    if dropWord:
        derivedFrom.discard(word.lower())
    if useCache:
        cacheDerivedFrom[word] = derivedFrom
    return derivedFrom, False

## LCS ##
def getAmbiguousWithLCS(useStemming, col1, columns):
    translate_table = dict((ord(char), None) for char in string.punctuation)
    ambiguousValues = set()     
    for col2 in columns:
        if (col1 != col2):
            lcs = getLCS(col1, col2)
            minLen = min(len(col1), len(col2))
            maxLen = max(len(col1), len(col2))
            if (2 * minLen) < maxLen:
                continue
            if (len(lcs) >= (0.5*minLen) and len(lcs) > 1):
                lcs = lcs.translate(translate_table)
                if (lcs.lower() in stopwords.words('english')):
                    continue
                if (useStemming):
                    words_in_lcs = word_tokenize(lcs)
                    lcs_stemmed = ""
                    for w in words_in_lcs:
                        stemmedW = ps.stem(w)
                        if (stemmedW in stemmed_words_list):
                            lcs_stemmed += stemmedW + " "
                    if (len(lcs_stemmed) > 0):
                        #print(col1, col2, lcs, lcs_stemmed, sep="\t")
                        ambiguousValues.add(lcs)
                else:
                    #print(col1, col2, lcs, sep="\t")
                    ambiguousValues.add(lcs)
    return ambiguousValues

## WIKIPEDIA ##
def getAmbiguityFromWikipedia(column, useStemming=True, results=2, useCache=True):
        if useCache and column in cacheWikipedia:
            return cacheWikipedia[column], True
        wikipediaResults = set()
        try:
            wikipediaResults = set(wikipedia.search(column.replace("-"," "), results))
            wikipediaResultsStrip = set()
            for result in wikipediaResults:
                wikipediaResultsStrip.add(result.lower().strip())
            wikipediaResults = wikipediaResultsStrip
        except Exception:
                pass
        if (useStemming):
            translate_table = dict((ord(char), None) for char in string.punctuation)
            columnNoPunct = column.translate(translate_table)
            stemmed_columns = [ps.stem(w) for w in word_tokenize(columnNoPunct)]
            wikipediaResultsStemmed = set()
            for wikipediaResult in wikipediaResults:
                wikipediaResult = wikipediaResult.translate(translate_table)
                words_in_wiki = word_tokenize(wikipediaResult)
                stemmed_wiki = ""
                for w in words_in_wiki:
                    w_stemmed = ps.stem(w)
                    if w_stemmed in stemmed_columns:
                        stemmed_wiki = w + " "
                if (len(stemmed_wiki) > 0):
                    wikipediaResultsStemmed.add(stemmed_wiki.lower().strip())
            if useCache:
                cacheWikipedia[column] = wikipediaResultsStemmed
            return wikipediaResultsStemmed, True
        else:
            if useCache:
                cacheWikipedia[column] = wikipediaResults
            return wikipediaResults, False

## FUNCTIONS ##

def getLabel(columns, limit=10, useStemming=True, useLCS=False):
    labels = {}
    provenance = {}
    for column in columns:
        provenanceMap = {}
        #start_time = time.time()
        synonyms, cachedSynonyms = getSynonym(column.replace("-"," "), limit=limit, dropWord=True, useCache=True)
        #print("Synonyms time: %s" %(time.time() - start_time))
        #start_time = time.time()
        relatedTo, cachedrelatedTo = getRelatedTo(column.replace("-"," "), limit=limit, dropWord=True, useCache=True)
        #print("RelatedTo time: %s" %(time.time() - start_time))
        #start_time = time.time()
        isA, cachedIsA = getIsA(column.replace("-"," "), limit=limit, dropWord=True, useCache=True)
        #print("IsA time: %s" %(time.time() - start_time))
        #start_time = time.time()
        derivedFrom, cachedDerivedFrom = getDerivedFrom(column.replace("-"," "), limit=limit, dropWord=True, useCache=True)
        #print("DerivedFrom time: %s" %(time.time() - start_time))
        #start_time = time.time()
        #wikipediaResults = set(wikipedia.search(column.replace("-"," "), results=2))
        wikipediaResults, cachedWikipediaResults = getAmbiguityFromWikipedia(column.replace("-"," "), useStemming=True, results=2, useCache=True)
        #print("Wikipedia time: %s" %(time.time() - start_time))
        #start_time = time.time()
        lcsAmb = getAmbiguousWithLCS(useStemming, column, columns)
        #print("LSC time: %s" %(time.time() - start_time))
        labelsForColumn = set()
        if len(synonyms) > 0:
            labelsForColumn = labelsForColumn | set(synonyms)
        if len(relatedTo) > 0:
            labelsForColumn = labelsForColumn | set(relatedTo)
        if len(wikipediaResults) > 0:
            labelsForColumn = labelsForColumn | set(wikipediaResults)
        if len(isA) > 0:
            labelsForColumn = labelsForColumn | set(isA)
        if len(derivedFrom) > 0:
            labelsForColumn = labelsForColumn | set(derivedFrom)
        #labelsForColumn = synonyms | relatedTo | wikipediaResults | isA | derivedFrom
        if len(labelsForColumn) == 0:
            labelsForColumn = labelsForColumn | lcsAmb
        if useLCS:
            labelsForColumn = labelsForColumn | lcsAmb
        provenanceMap['synonyms'] = synonyms
        provenanceMap['relatedTo'] = relatedTo
        provenanceMap['isA'] = isA
        provenanceMap['derivedFrom'] = derivedFrom
        provenanceMap['wikipediaResults'] = wikipediaResults
        provenanceMap['lcsAmb'] = lcsAmb
        labelsForColumn = [w.lower().strip() for w in list(labelsForColumn)]
        labels[column] = set(labelsForColumn)
        provenance[column] = provenanceMap
        if not (cachedSynonyms or cachedrelatedTo or cachedIsA or cachedDerivedFrom):
            time.sleep(1)
    return labels, provenance

def getAmbiguousColumns(columns, aliasDict):
    colSet = set()
    ambiguousLabels = {}
    ambiguousAttr = {}
    for col1 in columns:
        for col2 in columns:
            if col1 != col2:
                alias1 = aliasDict[col1]
                alias2 = aliasDict[col2]
                ambiguousValues = set(alias1).intersection(set(alias2))
                if (len(ambiguousValues) > 0):
                    colSet.add(col1)
                    colSet.add(col2)
                    if col1 not in ambiguousLabels:
                        ambiguousLabels[col1] = set(ambiguousValues)
                    else:
                        setValue = ambiguousLabels[col1]
                        setValue = setValue | ambiguousValues
                        ambiguousLabels[col1] = setValue
                    if col2 not in ambiguousLabels:
                        ambiguousLabels[col2] = set(ambiguousValues)
                    else:
                        setValue = ambiguousLabels[col2]
                        setValue = setValue | ambiguousValues
                        ambiguousLabels[col2] = setValue
                    if col1 not in ambiguousAttr:
                        setAttr = set()
                        ambiguousAttr[col1] = setAttr
                    setAttr = ambiguousAttr[col1]
                    setAttr.add(col2)
    return colSet, ambiguousLabels, ambiguousAttr

def updateAlias(dic, dicAdd, dicRemove, blackList = set()):
    for key, value in dicAdd.items():
        if key not in dic:
            dic[key] = set(value)
        else:
            setValue = dic[key]
            for v in value:
                setValue.add(v)
    for key, value in dicRemove.items():
        if key in dic:
            setValue = dic[key]
            for v in value:
                setValue.discard(v)
    #print("BLACKLIST:")
    #print(blackList)
    for key, value in dic.items():
        #print("BEFORE:")
        #print(value)
        value = value - blackList
        dic[key] = value
        #print("AFTER:")
        #print(value)
    return dic


In [9]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

def getStatsForValues(df, columns): 
    numerical = {}
    categorical = {}
    text = {}
    for column in columns:
        df[column] = pd.to_numeric(df[column], errors='ignore')
        columnType = df[column].dtype
        if (columnType in numerics):
            stats = df[column].describe()
            min = stats['min']
            max = stats['max']
            mean = stats['mean']
            median = stats['50%']
            std = stats['std']
            stats = (min, max, mean, median, std)
            numerical[column] = stats
        else:
            lenValue = df[column].nunique()
            lenColumn = df[column].shape[0]
            threshold = lenColumn * 0.9
            if (lenValue < threshold):
                #compute frequencies
                freq = df[column].value_counts()
                categorical[column] = freq
            else:
                mean = df[column].map(len).mean()
                std = df[column].map(len).std()
                median = df[column].map(len).median()
                tmpCol = df[column].sort_values()
                min = tmpCol.iloc[0]
                max = tmpCol.iloc[-1]
                stats = (min, max, mean, median, std)
                numerical[column] = stats

    return numerical, categorical, text

In [10]:
def findAmbiguousForCategorical(categorical):
    colSet = set()
    ambiguousValuesForAttr = {}
    setPairs = {}
    for c1, freq1 in categorical.items():
        for c2, freq2 in categorical.items():
            if (c1 != c2):
                keys1 = freq1.keys()
                keys2 = freq2.keys()
                ambiguousValues = set(keys1).intersection(set(keys2))
                if (len(ambiguousValues) < 1):
                    continue
                colSet.add(c1)
                colSet.add(c2)
                if c1 not in ambiguousValuesForAttr:
                    ambiguousValuesForAttr[c1] = set(ambiguousValues)
                else:
                    setValue = ambiguousValuesForAttr[c1]
                    setValue = setValue | ambiguousValues
                    ambiguousValuesForAttr[c1] = setValue
                if c1 not in setPairs:
                    attrs = set()
                    attrs.add(c2)
                    setPairs[c1] = attrs
                else:
                    attrs = setPairs[c1]
                    attrs.add(c2)
    return colSet, ambiguousValuesForAttr, setPairs

def getIntersection(min1, max1, min2, max2):
    ## check type
    if (type(min1) == str) or (type(max1) == str) or (type(min2) == str) or (type(max2) == str):
        return None
    ## check intervals
    if (max1 < min1) or (max2 < min2):
        print("Error min1-max1: {}-{} min2-max2:{}-{}".format(min1, max1, min2, max2))
        return None
    # case min1 - max1, min2 - max2 --> no intersection
    if (max1 < min2):
        return None
    # case min2 - max2, min1 - max2 --> no intersection
    if (max2 < min1):
        return None
    # case min1, min2, max1, max2 --> min2-max1
    if (min1 <= min2) and (min2 <= max1) and (max1 <= max2):
        return (min2, max1)
    # case min2, min1, max2, max1 --> min2-max1
    if (min2 <= min1) and (min1 <= max2) and (max2 <= max1):
        return (min1, max2)
    # case min1, min2, max2, max1 --> min2-max2
    if (min1 <= min2) and (min2 <= max2) and (max2 <= max1):
        return (min2, max2)
    # case min2, min1, max1, max2 --> min1-max1
    return (min1, max1)


def findAmbiguousForNumerical(numerical):
    ## naive strategy: intersection of intervals
    colSet = set()
    ambiguousValuesForAttr = {}
    setPairs = {}
    for c1, stats1 in numerical.items():
        for c2, stats2 in numerical.items():
            if (c1 != c2):
                interval = getIntersection(stats1[0], stats1[1], stats2[0], stats2[1])
                if (interval is None):
                    continue
                colSet.add(c1)
                colSet.add(c2)
                if c1 not in ambiguousValuesForAttr:
                    listValue = []
                    listValue.append(interval)
                    ambiguousValuesForAttr[c1] = listValue
                else:
                    listValue = ambiguousValuesForAttr[c1]
                    listValue.append(interval)
                if c1 not in setPairs:
                    attrs = set()
                    attrs.add(c2)
                    setPairs[c1] = attrs
                else:
                    attrs = setPairs[c1]
                    attrs.add(c2)
    return colSet, ambiguousValuesForAttr, setPairs

In [11]:
def findAmbiguousForLabelAndData(setPairs, setPairsCategorical, setPairsNumerical, ambiguousValuesForAttrCategorical, ambiguousValuesForAttrNumerical):
    setPairsLabelData = {}
    ambiguousValues = {}
    for attributeLabel, ambiguousAttributesLabel in setPairs.items():
        setAmb = set()
        for attributeCategorical, ambiguousAttributesCategorical in setPairsCategorical.items():
            if (attributeLabel == attributeCategorical):
                intersection = ambiguousAttributesLabel.intersection(ambiguousAttributesCategorical)
                #intersection = list(intersection)
                if (len(intersection) > 0):
                    for amb in intersection:
                        setAmb.add(amb)
                    for attr in intersection:
                        valuesAmb = ambiguousValuesForAttrCategorical[attr]
                        for value in valuesAmb:
                            if value in ambiguousValues:
                                setAttrs = ambiguousValues[value]
                                setAttrs.add(attr)
                            else:
                                setAttrs = set()
                                setAttrs.add(attr)
                                ambiguousValues[value] = setAttrs
        for attributeNumerical, ambiguousAttributesNumerical in setPairsNumerical.items():
            if (attributeLabel == attributeNumerical):
                intersection = ambiguousAttributesLabel.intersection(ambiguousAttributesNumerical)
                #intersection = list(intersection)
                if (len(intersection) > 0):
                    for amb in intersection:
                        setAmb.add(amb)
                    print(intersection)
                    for attr in intersection:
                        intervals = ambiguousValuesForAttrNumerical[attr]
                        for interval in intervals:
                            minI = interval[0]
                            maxI = interval[1]
                            if minI in ambiguousValues:
                                setAttrs = ambiguousValues[minI]
                                setAttrs.add(attr)
                            else:
                                setAttrs = set()
                                setAttrs.add(attr)
                                ambiguousValues[minI] = setAttrs
                            if maxI in ambiguousValues:
                                setAttrs = ambiguousValues[maxI]
                                setAttrs.add(attr)
                            else:
                                setAttrs = set()
                                setAttrs.add(attr)
                                ambiguousValues[maxI] = setAttrs
        setPairsLabelData[attributeLabel] = setAmb
    return setPairsLabelData, ambiguousValues

def reversedMap(map):
    reversed = {}
    for attrName, values in map.items():
        for value in values:
            #print(attrName, value)
            if (value not in reversed):
                    attrSet = set()
                    attrSet.add(attrName)
                    reversed[value] = attrSet
            else:
                    attrSet = reversed[value]
                    attrSet.add(attrName)
            #if type(value) is tuple:
            #    print("TUPLE")    
    return reversedMap

# Data retrieve

In [12]:
## WDC download
## one time execution
skipDownload = False
if skipDownload == False:
    #for i in range(0, 25):
    for i in range(3, 4):    
        s = str(i)
        if i < 10:
            s = '0' + str(i)
        url = "http://data.dws.informatik.uni-mannheim.de/webtables/2015-07/englishCorpus/compressed/" + s + ".tar.gz"
        response = requests.get(url, stream=True)
        file = tarfile.open(fileobj=response.raw, mode="r|gz")
        path = "./" + s + "/"
        file.extractall(path=path)
        file.close()
        tarFile = "./"+s+"/"+s+".tar"
        !python -m tarfile -e {tarFile} ./WDC/

        #os.remove(path)
        os.remove(tarFile)
        #os.remove('./WDC/')

In [13]:
## WDC Web Table Corpus 2015
## All data
relationalTables = []

#path_to_json = './WDC/0'
#path_to_json = './WDC/1'
#path_to_json = './WDC/2'
path_to_json = './WDC/3'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

start_time = time.time()
for json_file in json_files:
    path = path_to_json + "/" + json_file
    with open(path) as f:
        data = json.load(f)
        if (data['tableType'] == 'RELATION') and (data['hasHeader'] == True) and (data['tableOrientation'] == "HORIZONTAL") and (data['headerPosition'] == "FIRST_ROW"):
            relationalTables.append(data)
end_time = time.time()
elapsed_time = end_time - start_time
print("Loaded Relational JSON Files:", len(relationalTables))
print("Loaded in ", elapsed_time)

#parsedSchemas = {}
parsedSchemas = []
start_time = time.time()
for table in relationalTables:
#for table in relationalTables[:5000]:
    relationData = table['relation']
    #pageTitle = table['pageTitle'].strip()
    #title = table['title'].strip()
    #print("PageTitle:", pageTitle)
    #print("Title:",title)
    columns = []
    data = {}
    for column in relationData:
        columnName = column[0].strip()
        if columnName != '':
            columns.append(columnName)
            data[columnName] = column[1:]
    if len(data) > 0:
        t = (columns, data)
        parsedSchemas.append(t)

print("Parsed schemas: ", len(parsedSchemas))
end_time = time.time()
elapsed_time = end_time - start_time
print("Parsed in ", elapsed_time)
del relationalTables

Loaded Relational JSON Files: 891061
Loaded in  732.6717350482941
Parsed schemas:  888515
Parsed in  27.361342906951904


In [ ]:
structuredData = []
#K = 1500000
#K = 1000
#K = 5000
#K = 10000
#K = 25000
#K = 50000
#K = 75000
#K = 100000
#K = 150000
K = 225000
#K = 400000
#K = 500000
#K = 750000
#K = 1000000
#K = 1500000

stats = {"tables": 0, "columns": 0, 'synonyms':0, 'relatedTo':0, 'isA':0, 'derivedFrom':0, 'wikipediaResults':0, 'lcsAmb':0}

#for table, schemaAndData in parsedSchemas.items():
#for schemaAndData in parsedSchemas:
count = 0
start_time = time.time()
for schemaAndData in parsedSchemas:
    columns = schemaAndData[0]
    #print(columns)
    aliasDict, provenance, colsToFind = findFromCached(cachedAlias, cachedProvenance, columns)
    if (len(colsToFind) > 0):
        #search online
        #print("Search online since not stored in the cache for alias")
        aliasDictOnline, provenanceOnline = getLabel(colsToFind)
        #print(aliasDictOnline)
        #print(provenanceOnline)
        aliasDict.update(aliasDictOnline)
        provenance.update(provenanceOnline)
        cachedAlias.update(aliasDict)
        cachedProvenance.update(provenance)
    #aliasDict, provenance = getLabel(columns)
    #print(aliasDict)
    #print(provenance)
    ## TODO: periodically save caches
    colSet, ambiguousLabels, setPairs = getAmbiguousColumns(columns, aliasDict)
    #print(colSet)
    #print(ambiguousLabels)
    #print(setPairs)
    #print(provenance)
    colAmbiguous = list(colSet)
    count += 1
    if (count % 100) == 0:
        print("Processed: ", count)
        print("Elapsed time: %s" %(time.time() - start_time))
        print("StructuredData size: ", len(structuredData))
    if (len(colSet) > 0):
        stats['tables'] = stats['tables'] + 1
        stats['columns'] = stats['columns'] + len(columns)
        table = schemaAndData[1]
        t = (table, columns, setPairs, ambiguousLabels, provenance)
        structuredData.append(t)
    if len(structuredData) == K:
        break
print("Process ended in time: %s" %(time.time() - start_time))
print("Structured data: ", len(structuredData))

Processed:  100
Elapsed time: 0.021344900131225586
StructuredData size:  36
Processed:  200
Elapsed time: 0.03374505043029785
StructuredData size:  66
Processed:  300
Elapsed time: 0.04154682159423828
StructuredData size:  92
Processed:  400
Elapsed time: 0.052993059158325195
StructuredData size:  117
Processed:  500
Elapsed time: 0.06653904914855957
StructuredData size:  148
Processed:  600
Elapsed time: 0.0747981071472168
StructuredData size:  179
Processed:  700
Elapsed time: 0.08206820487976074
StructuredData size:  208
Processed:  800
Elapsed time: 0.08974194526672363
StructuredData size:  240
Processed:  900
Elapsed time: 0.09743618965148926
StructuredData size:  268
Processed:  1000
Elapsed time: 0.1042330265045166
StructuredData size:  298
Processed:  1100
Elapsed time: 0.11251497268676758
StructuredData size:  330
Processed:  1200
Elapsed time: 0.12050819396972656
StructuredData size:  361
Processed:  1300
Elapsed time: 0.12638092041015625
StructuredData size:  389
Processed: 

Processed:  12800
Elapsed time: 0.9057979583740234
StructuredData size:  3724
Processed:  12900
Elapsed time: 0.9110360145568848
StructuredData size:  3750
Processed:  13000
Elapsed time: 0.9172871112823486
StructuredData size:  3782
Processed:  13100
Elapsed time: 0.9241209030151367
StructuredData size:  3809
Processed:  13200
Elapsed time: 0.9310550689697266
StructuredData size:  3845
Processed:  13300
Elapsed time: 0.9381570816040039
StructuredData size:  3869
Processed:  13400
Elapsed time: 0.9457781314849854
StructuredData size:  3900
Processed:  13500
Elapsed time: 0.954064130783081
StructuredData size:  3929
Processed:  13600
Elapsed time: 0.9602029323577881
StructuredData size:  3956
Processed:  13700
Elapsed time: 0.966357946395874
StructuredData size:  3977
Processed:  13800
Elapsed time: 0.9726991653442383
StructuredData size:  4003
Processed:  13900
Elapsed time: 0.9823770523071289
StructuredData size:  4034
Processed:  14000
Elapsed time: 0.9901080131530762
StructuredData 

Processed:  26400
Elapsed time: 1.810006856918335
StructuredData size:  7710
Processed:  26500
Elapsed time: 1.8156919479370117
StructuredData size:  7740
Processed:  26600
Elapsed time: 1.8239500522613525
StructuredData size:  7777
Processed:  26700
Elapsed time: 1.8327341079711914
StructuredData size:  7802
Processed:  26800
Elapsed time: 1.8407089710235596
StructuredData size:  7838
Processed:  26900
Elapsed time: 1.8503668308258057
StructuredData size:  7871
Processed:  27000
Elapsed time: 1.8584108352661133
StructuredData size:  7898
Processed:  27100
Elapsed time: 1.8673350811004639
StructuredData size:  7927
Processed:  27200
Elapsed time: 1.8813159465789795
StructuredData size:  7966
Processed:  27300
Elapsed time: 1.8919720649719238
StructuredData size:  7992
Processed:  27400
Elapsed time: 1.9015870094299316
StructuredData size:  8024
Processed:  27500
Elapsed time: 1.914815902709961
StructuredData size:  8060
Processed:  27600
Elapsed time: 1.9223930835723877
StructuredData 

Processed:  39100
Elapsed time: 2.7116189002990723
StructuredData size:  11448
Processed:  39200
Elapsed time: 2.716620922088623
StructuredData size:  11477
Processed:  39300
Elapsed time: 2.7225749492645264
StructuredData size:  11510
Processed:  39400
Elapsed time: 2.7280800342559814
StructuredData size:  11535
Processed:  39500
Elapsed time: 2.7354788780212402
StructuredData size:  11563
Processed:  39600
Elapsed time: 2.742340087890625
StructuredData size:  11590
Processed:  39700
Elapsed time: 2.748440980911255
StructuredData size:  11619
Processed:  39800
Elapsed time: 2.7579381465911865
StructuredData size:  11645
Processed:  39900
Elapsed time: 2.7672648429870605
StructuredData size:  11676
Processed:  40000
Elapsed time: 2.772883176803589
StructuredData size:  11703
Processed:  40100
Elapsed time: 2.778503179550171
StructuredData size:  11730
Processed:  40200
Elapsed time: 2.7834019660949707
StructuredData size:  11756
Processed:  40300
Elapsed time: 2.7910470962524414
Struct

Processed:  51700
Elapsed time: 3.584033966064453
StructuredData size:  15071
Processed:  51800
Elapsed time: 3.590167999267578
StructuredData size:  15102
Processed:  51900
Elapsed time: 3.594583034515381
StructuredData size:  15132
Processed:  52000
Elapsed time: 3.5995471477508545
StructuredData size:  15154
Processed:  52100
Elapsed time: 3.6052188873291016
StructuredData size:  15180
Processed:  52200
Elapsed time: 3.6142470836639404
StructuredData size:  15207
Processed:  52300
Elapsed time: 3.622281074523926
StructuredData size:  15234
Processed:  52400
Elapsed time: 3.627763032913208
StructuredData size:  15255
Processed:  52500
Elapsed time: 3.6361560821533203
StructuredData size:  15281
Processed:  52600
Elapsed time: 3.646313190460205
StructuredData size:  15314
Processed:  52700
Elapsed time: 3.655210018157959
StructuredData size:  15347
Processed:  52800
Elapsed time: 3.660836935043335
StructuredData size:  15371
Processed:  52900
Elapsed time: 3.668694019317627
Structured

...

Processed:  681000
Elapsed time: 42.19251608848572
StructuredData size:  198814
Processed:  681100
Elapsed time: 42.19773197174072
StructuredData size:  198848
Processed:  681200
Elapsed time: 42.20372486114502
StructuredData size:  198874
Processed:  681300
Elapsed time: 42.2078800201416
StructuredData size:  198901
Processed:  681400
Elapsed time: 42.21318697929382
StructuredData size:  198931
Processed:  681500
Elapsed time: 42.21952700614929
StructuredData size:  198953
Processed:  681600
Elapsed time: 42.22545599937439
StructuredData size:  198984
Processed:  681700
Elapsed time: 42.23072099685669
StructuredData size:  199014
Processed:  681800
Elapsed time: 42.23810005187988
StructuredData size:  199039
Processed:  681900
Elapsed time: 42.245235204696655
StructuredData size:  199070
Processed:  682000
Elapsed time: 42.25021719932556
StructuredData size:  199100
Processed:  682100
Elapsed time: 42.25436091423035
StructuredData size:  199128
Processed:  682200
Elapsed time: 42.2592

Processed:  691300
Elapsed time: 2077.2267949581146
StructuredData size:  201779
Processed:  691400
Elapsed time: 2077.2332739830017
StructuredData size:  201805
Processed:  691500
Elapsed time: 2089.095920085907
StructuredData size:  201838
Processed:  691600
Elapsed time: 2132.524610042572
StructuredData size:  201869
Processed:  691700
Elapsed time: 2148.148003101349
StructuredData size:  201895
Processed:  691800
Elapsed time: 2168.034768104553
StructuredData size:  201929
Processed:  691900
Elapsed time: 2176.0180928707123
StructuredData size:  201955
Processed:  692000
Elapsed time: 2176.026381969452
StructuredData size:  201989
Processed:  692100
Elapsed time: 2187.9644300937653
StructuredData size:  202022
Processed:  692200
Elapsed time: 2215.101491212845
StructuredData size:  202051
Processed:  692300
Elapsed time: 2244.0255711078644
StructuredData size:  202080
Processed:  692400
Elapsed time: 2265.6253831386566
StructuredData size:  202107
Processed:  692500
Elapsed time: 2

Processed:  701500
Elapsed time: 4777.400134086609
StructuredData size:  204765
Processed:  701600
Elapsed time: 4799.527209997177
StructuredData size:  204792
Processed:  701700
Elapsed time: 4822.961178064346
StructuredData size:  204819
Processed:  701800
Elapsed time: 4857.4860599040985
StructuredData size:  204848
Processed:  701900
Elapsed time: 4889.505513191223
StructuredData size:  204872
Processed:  702000
Elapsed time: 4901.231145858765
StructuredData size:  204903
Processed:  702100
Elapsed time: 4931.234925031662
StructuredData size:  204931
Processed:  702200
Elapsed time: 4968.752761125565
StructuredData size:  204956
Processed:  702300
Elapsed time: 4992.328101158142
StructuredData size:  204986
Processed:  702400
Elapsed time: 5031.187986135483
StructuredData size:  205018
Processed:  702500
Elapsed time: 5062.335994958878
StructuredData size:  205044
Processed:  702600
Elapsed time: 5094.439190864563
StructuredData size:  205074
Processed:  702700
Elapsed time: 5133.8

Processed:  711800
Elapsed time: 7565.300757884979
StructuredData size:  207789
Processed:  711900
Elapsed time: 7597.8948221206665
StructuredData size:  207828
Processed:  712000
Elapsed time: 7622.096364021301
StructuredData size:  207861
Processed:  712100
Elapsed time: 7642.677891969681
StructuredData size:  207890
Processed:  712200
Elapsed time: 7682.033065080643
StructuredData size:  207920
Processed:  712300
Elapsed time: 7689.67386507988
StructuredData size:  207956
Processed:  712400
Elapsed time: 7701.360718011856
StructuredData size:  207989
Processed:  712500
Elapsed time: 7726.103098154068
StructuredData size:  208020
Processed:  712600
Elapsed time: 7741.534651994705
StructuredData size:  208051
Processed:  712700
Elapsed time: 7761.581084012985
StructuredData size:  208079
Processed:  712800
Elapsed time: 7793.341367006302
StructuredData size:  208104
Processed:  712900
Elapsed time: 7825.811705112457
StructuredData size:  208127
Processed:  713000
Elapsed time: 7876.94

Processed:  722100
Elapsed time: 10129.09151005745
StructuredData size:  210777
Processed:  722200
Elapsed time: 10190.709495067596
StructuredData size:  210801
Processed:  722300
Elapsed time: 10222.898158073425
StructuredData size:  210836
Processed:  722400
Elapsed time: 10231.731209039688
StructuredData size:  210874
Processed:  722500
Elapsed time: 10247.049479007721
StructuredData size:  210900
Processed:  722600
Elapsed time: 10254.762815952301
StructuredData size:  210924
Processed:  722700
Elapsed time: 10298.202420949936
StructuredData size:  210952
Processed:  722800
Elapsed time: 10317.475070238113
StructuredData size:  210979
Processed:  722900
Elapsed time: 10341.901422023773
StructuredData size:  211009
Processed:  723000
Elapsed time: 10393.41222500801
StructuredData size:  211038
Processed:  723100
Elapsed time: 10417.436152935028
StructuredData size:  211069
Processed:  723200
Elapsed time: 10421.775515079498
StructuredData size:  211095
Processed:  723300
Elapsed tim

In [ ]:
## caches
start_time = time.time()
saveCache('./cacheSynonym-small.json', cacheSynonym)
saveCache('./cacheRelatedTo-small.json', cacheRelatedTo)
saveCache('./cacheIsA-small.json', cacheIsA)
saveCache('./cacheDerivedFrom-small.json', cacheDerivedFrom)
saveCache('./cacheWikipedia-small.json', cacheWikipedia)
saveCache('./cacheAlias.json', cachedAlias)
total_time = time.time() - start_time
print("Process ended in time:", total_time)

In [ ]:
a_file = open("cacheProvenance.pkl", "wb")
pickle.dump(cachedProvenance, a_file)
a_file.close()

In [ ]:
def generateRow(schemaString, attr1, attr2, valueString):
    return schemaString + " attr1: " + attr1 + " attr2: " + attr2 + "\t" + valueString+"\n"

start_time = time.time()
examples = []
distinctLabels = set()
distinctForType = {}
for t in structuredData:
    table = t[0]
    columns = t[1]
    setPairs = t[2]
    ambiguousLabels = t[3]
    provenance = t[4]
    schemaString = "|".join(columns)
    #TODO: implement it
    #schemaString = generateFromData(table, columns)
    countGenerated = 0
    for attr1, attrSet in setPairs.items():
        for attr2 in attrSet:
            amb1 = ambiguousLabels[attr1]
            amb2 = ambiguousLabels[attr2]
            ambValues = set(amb1).intersection(set(amb2))
            ambValues.discard(attr1.lower())
            ambValues.discard(attr2.lower())
            prov1 = provenance[attr1]
            prov2= provenance[attr2]
            for ambValue in ambValues:
                distinctLabels.add(ambValue)
                for key, valueSet in prov1.items():
                    if ambValue in valueSet:
                        stats[key] = stats[key] + 1
                        if key not in distinctForType:
                            distinctForType[key] = set()
                        distLabelType = distinctForType[key]
                        distLabelType.add(ambValue)
                for key, valueSet in prov2.items():
                    if ambValue in valueSet:
                        stats[key] = stats[key] + 1
                        if key not in distinctForType:
                            distinctForType[key] = set()
                        distLabelType = distinctForType[key]
                        distLabelType.add(ambValue)
                example = generateRow(schemaString, attr1, attr2, ambValue)
                examples.append(example)
                countGenerated += 1
    for i in range(0, countGenerated):
        #TODO improve the column selection
        attr1 = choice(list(columns))
        attr2 = choice(list(columns))
        if (attr1 not in setPairs):
            example = generateRow(schemaString, attr1, attr2, "None")
            examples.append(example)
        else:
            attr1AmbSet = setPairs[attr1]
            if ((attr2 != attr1) and (attr2 not in attr1AmbSet)):
                example = generateRow(schemaString, attr1, attr2, "None")
                examples.append(example)
                
print(stats)
print("**** DISTINCT *****")
for key, value in distinctForType.items():
    print(key, len(value))
print("Unique labels: ", len(distinctLabels))
total_time = time.time() - start_time
print("Process ended in time:", total_time)

In [ ]:
print("Examples:", len(examples))
examplesNoDuplicates = set(examples)
print("Examples no duplicates:", len(examplesNoDuplicates))
start_time = time.time()
fileName = "./generated/trainWDC-00-" + str(K) + "-v1.3.tsv"
#f = open("./trainWDC-00-5k-v1.1.tsv", "w")
f = open(fileName, "w")
#f.writelines(examples)
f.writelines(examplesNoDuplicates)
f.close()
total_time = time.time() - start_time
print("Process ended in time:", total_time)